<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/final_project05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
#from google.colab import files
#uploaded=files.upload()

In [ ]:
#from google.colab import files
#uploaded=files.upload()

In [ ]:
양파_특=pd.read_excel('/content/양파_특(20kg)_dt.xlsx')
양파_상=pd.read_excel('/content/양파_상(20kg)_dt.xlsx')
배추_특=pd.read_excel('/content/배추_특(10kg)_dt.xlsx')
배추_상=pd.read_excel('/content/배추_상(10kg)_dt.xlsx')

In [ ]:
del 양파_특['일자']

In [ ]:
양파_특.head()

In [ ]:
양파_특=양파_특[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량']]

In [ ]:
양파_특corr = 양파_특.corr(method = 'pearson') 
양파_특corr

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
# 코랩 한글깨짐 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
#mpl.font_manager._rebuild()

In [ ]:
df_heatmap = sns.heatmap(양파_특corr, cbar = True, annot = True, annot_kws={'size' : 10}, fmt = '.2f', square = True, cmap = 'Blues')
df_heatmap


In [ ]:
import scipy.stats as ss 

양파_특_ss=ss.zscore(양파_특)
print(양파_특_ss)

In [ ]:
pd.DataFrame(양파_특_ss)

In [ ]:
feature_columns=list(양파_특.columns.difference(['평균가격']))
X=양파_특[feature_columns]
y=양파_특.평균가격
train_x,test_x,train_y,test_y=train_test_split(X,y,train_size=0.7,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
#Train the MLR/회귀모델적합
full_model=sm.OLS(train_y,train_x)
fitted_full_model=full_model.fit()

In [ ]:
fitted_full_model.summary()

In [ ]:
# VIF를 통한 다중공선성 확인 
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_특.values,i) for i in range(양파_특.shape[1])]
vif['features']=양파_특.columns
vif

In [ ]:
# 변수선택법 
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)]) # modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {'model' : regr, 'AIC' : AIC}

print(processSubset(X=train_x, y = train_y, feature_set = feature_columns[0:5]))

In [ ]:
# 전진선택법(step=1)

def forward(X, y, predictors):
    # 데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predcitors+[p]+['const']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:',best_model[0])
    return best_model

In [ ]:
# 전진선택법 모델
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=['AID', 'model'])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0 ~ 9  -> 1 ~ 10
    for i in range(1, len(X.columns.difference(['const'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [ ]:
#Forward_best_model = forward_model(X=train_x, y=train_y)